## 📦 Instalação de Dependências

In [ ]:
# Instalar biblioteca requests para fazer chamadas HTTP
!pip install requests -q

In [ ]:
import requests
import json
from IPython.display import display, Markdown, HTML
import time

## 🛠️ Funções Auxiliares

In [ ]:
def mostrar_resposta(metodo, endpoint, response):
    """Mostra a resposta da API de forma formatada"""
    cor = "green" if 200 <= response.status_code < 300 else "red"
    
    html = f"""
    <div style="border: 2px solid {cor}; padding: 10px; margin: 10px 0; border-radius: 5px;">
        <h4 style="color: {cor}; margin: 0;">{metodo} {endpoint}</h4>
        <p><strong>Status:</strong> {response.status_code}</p>
    """
    
    try:
        dados = response.json()
        html += f"<pre>{json.dumps(dados, indent=2, ensure_ascii=False)}</pre>"
    except:
        html += f"<p>{response.text}</p>"
    
    html += "</div>"
    display(HTML(html))
    return response

def testar_conexao(url):
    """Testa se a API está respondendo"""
    try:
        response = requests.get(url, timeout=2)
        return True
    except:
        return False

## ⚙️ Configuração do Framework

**Escolha qual framework testar:**

In [ ]:
# Configurações de cada framework
FRAMEWORKS = {
    "fastapi": {"url": "http://localhost:8000", "cmd": "uvicorn fastapi_app:app --reload"},
    "flask": {"url": "http://localhost:5000", "cmd": "python flask_app.py"},
    "sanic": {"url": "http://localhost:8000", "cmd": "python sanic_app.py"},
    "tornado": {"url": "http://localhost:8888", "cmd": "python tornado_app.py"},
    "falcon": {"url": "http://localhost:8000", "cmd": "uvicorn falcon_app:app"},
    "bottle": {"url": "http://localhost:8080", "cmd": "python bottle_app.py"},
}

# ALTERE AQUI o framework que deseja testar
FRAMEWORK_ATUAL = "fastapi"  # Opções: fastapi, flask, sanic, tornado, falcon, bottle

BASE_URL = FRAMEWORKS[FRAMEWORK_ATUAL]["url"]
COMANDO = FRAMEWORKS[FRAMEWORK_ATUAL]["cmd"]

print(f"🎯 Framework selecionado: {FRAMEWORK_ATUAL.upper()}")
print(f"🌐 URL: {BASE_URL}")
print(f"\n⚠️  Para iniciar o servidor, execute em um terminal:")
print(f"   {COMANDO}")

# Verificar se está rodando
if testar_conexao(BASE_URL):
    print(f"\n✅ API está rodando!")
else:
    print(f"\n❌ API não está respondendo. Inicie o servidor primeiro!")

## 🧪 Teste 1: Endpoint Home (GET /)

In [ ]:
response = requests.get(f"{BASE_URL}/")
mostrar_resposta("GET", "/", response)

## 🧪 Teste 2: Criar Notas (POST /notas)

In [ ]:
# Criar primeira nota
nota1 = {
    "titulo": "Minha Primeira Nota",
    "conteudo": "Este é o conteúdo da primeira nota criada via Jupyter!"
}

response = requests.post(
    f"{BASE_URL}/notas",
    json=nota1,
    headers={"Content-Type": "application/json"}
)
mostrar_resposta("POST", "/notas", response)

# Salvar o ID da primeira nota
primeira_nota = response.json()
nota_id = primeira_nota.get("id", 1)

In [ ]:
# Criar mais notas para popular o banco
notas_exemplo = [
    {"titulo": "Lista de Compras", "conteudo": "Leite, Pão, Café, Frutas"},
    {"titulo": "Ideias de Projeto", "conteudo": "1. App mobile\n2. Dashboard analytics\n3. API Gateway"},
    {"titulo": "Reunião Importante", "conteudo": "Discutir roadmap Q1 2026 - 14h às 16h"},
]

for nota in notas_exemplo:
    response = requests.post(f"{BASE_URL}/notas", json=nota)
    print(f"✅ Nota '{nota['titulo']}' criada - Status: {response.status_code}")
    time.sleep(0.1)

## 🧪 Teste 3: Listar Todas as Notas (GET /notas)

In [ ]:
response = requests.get(f"{BASE_URL}/notas")
mostrar_resposta("GET", "/notas", response)

# Contar notas
dados = response.json()
if isinstance(dados, dict) and "notas" in dados:
    total = len(dados["notas"])
elif isinstance(dados, list):
    total = len(dados)
else:
    total = 0

print(f"\n📊 Total de notas: {total}")

## 🧪 Teste 4: Obter Nota Específica (GET /notas/{id})

In [ ]:
# Usar o ID da primeira nota criada
response = requests.get(f"{BASE_URL}/notas/{nota_id}")
mostrar_resposta("GET", f"/notas/{nota_id}", response)

## 🧪 Teste 5: Atualizar Nota (PUT /notas/{id})

In [ ]:
nota_atualizada = {
    "titulo": "Nota ATUALIZADA via Jupyter",
    "conteudo": "O conteúdo foi modificado com sucesso! 🎉"
}

response = requests.put(
    f"{BASE_URL}/notas/{nota_id}",
    json=nota_atualizada,
    headers={"Content-Type": "application/json"}
)
mostrar_resposta("PUT", f"/notas/{nota_id}", response)

In [ ]:
# Verificar se a atualização foi aplicada
response = requests.get(f"{BASE_URL}/notas/{nota_id}")
mostrar_resposta("GET", f"/notas/{nota_id} (verificação)", response)

## 🧪 Teste 6: Deletar Nota (DELETE /notas/{id})

In [ ]:
response = requests.delete(f"{BASE_URL}/notas/{nota_id}")
mostrar_resposta("DELETE", f"/notas/{nota_id}", response)

In [ ]:
# Verificar se a nota foi deletada
response = requests.get(f"{BASE_URL}/notas/{nota_id}")
mostrar_resposta("GET", f"/notas/{nota_id} (após delete)", response)

if response.status_code == 404:
    print("\n✅ Nota deletada com sucesso!")

## 🧪 Teste 7: Listagem Final

In [ ]:
response = requests.get(f"{BASE_URL}/notas")
mostrar_resposta("GET", "/notas (listagem final)", response)

dados = response.json()
if isinstance(dados, dict) and "notas" in dados:
    total = len(dados["notas"])
elif isinstance(dados, list):
    total = len(dados)
else:
    total = 0

print(f"\n📊 Total de notas após delete: {total}")

## 📊 Resumo dos Testes

Execute a célula abaixo para ver um resumo completo:

In [ ]:
resumo = f"""
# ✅ Testes Concluídos - {FRAMEWORK_ATUAL.upper()}

## Operações Testadas:
- ✅ GET / (Home)
- ✅ POST /notas (Criar nota)
- ✅ GET /notas (Listar todas)
- ✅ GET /notas/{{id}} (Obter específica)
- ✅ PUT /notas/{{id}} (Atualizar)
- ✅ DELETE /notas/{{id}} (Deletar)

## Framework Testado:
**{FRAMEWORK_ATUAL.upper()}** rodando em `{BASE_URL}`

## Próximos Passos:
1. Altere a variável `FRAMEWORK_ATUAL` na célula de configuração
2. Inicie o servidor correspondente
3. Execute todas as células novamente

## Frameworks Disponíveis:
"""

for nome, config in FRAMEWORKS.items():
    resumo += f"\n- **{nome}**: `{config['cmd']}`"

display(Markdown(resumo))

## 🔄 Teste Automatizado de Múltiplos Frameworks

**Avançado:** Execute esta célula para testar todos os frameworks que estiverem rodando:

In [ ]:
def testar_framework(nome, url):
    """Testa um framework específico"""
    print(f"\n{'='*60}")
    print(f"🧪 Testando {nome.upper()}")
    print(f"{'='*60}")
    
    if not testar_conexao(url):
        print(f"❌ {nome} não está rodando em {url}")
        return False
    
    try:
        # Criar nota
        nota = {"titulo": f"Teste {nome}", "conteudo": f"Criado via teste automatizado"}
        r1 = requests.post(f"{url}/notas", json=nota)
        print(f"✅ POST /notas - Status: {r1.status_code}")
        
        # Listar
        r2 = requests.get(f"{url}/notas")
        print(f"✅ GET /notas - Status: {r2.status_code}")
        
        # Obter
        nota_id = r1.json().get("id", 1)
        r3 = requests.get(f"{url}/notas/{nota_id}")
        print(f"✅ GET /notas/{nota_id} - Status: {r3.status_code}")
        
        # Atualizar
        r4 = requests.put(f"{url}/notas/{nota_id}", json={"titulo": "Atualizado", "conteudo": "OK"})
        print(f"✅ PUT /notas/{nota_id} - Status: {r4.status_code}")
        
        # Deletar
        r5 = requests.delete(f"{url}/notas/{nota_id}")
        print(f"✅ DELETE /notas/{nota_id} - Status: {r5.status_code}")
        
        print(f"\n✅ {nome.upper()} passou em todos os testes!")
        return True
        
    except Exception as e:
        print(f"❌ Erro ao testar {nome}: {e}")
        return False

# Testar todos os frameworks
resultados = {}
for nome, config in FRAMEWORKS.items():
    resultados[nome] = testar_framework(nome, config["url"])
    time.sleep(0.5)

# Resumo final
print(f"\n\n{'='*60}")
print("📊 RESUMO FINAL")
print(f"{'='*60}")
for nome, passou in resultados.items():
    status = "✅" if passou else "❌"
    print(f"{status} {nome.upper()}")

total_passou = sum(resultados.values())
print(f"\n🎯 {total_passou}/{len(resultados)} frameworks testados com sucesso!")